In [30]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

In [36]:
%time
ep = 1e-4
time = np.arange(0,10,ep)
L = 1*np.float32(np.logical_and(time>2,time<4))

kr = 1/0.3
kf = 1/0.6

Cm = 0.00144
gL = 1.44
Vl = -62
VR = 0

gR = 0.5
thresh0 = -55
reset = -62

tGamma = 0.2
delth = 0.2

gB = 0.2
gN = 0.1

def f(y,t):
    Rs,V = y
    dRs = kr*L[int(t/ep)]*(1-Rs)-kf*Rs
    dV = 1/Cm*(-gL*(V-Vl)-gR*(V-VR)*Rs -gB*(V-VR)- gN*np.random.normal()*(V-VR))
    return np.array([dRs,dV])

X = np.zeros((2,time.shape[0]))
wth = np.zeros(time.shape[0])
sp = np.zeros(time.shape[0])

X[:,0]= [0,-60]

for i in range(1,time.shape[0]):
    X[:,i] = X[:,i-1] + ep*f(X[:,i-1],time[i-1])
    wth[i] = wth[i-1]*np.exp(-ep/tGamma) 
    if X[1,i]>thresh0+wth[i]:
        X[1,i] = reset
        sp[i-1] = 1
        wth[i] = wth[i] + delth/tGamma

plt.figure(figsize=(15,3))
plt.plot(time[:],X[1,:],"-",label="RK4 Solution for y")
plt.scatter(time[sp>0],sp[sp>0]+thresh0,marker="|",s=10000)

plt.xlabel("t")
plt.ylabel("X")
plt.ylim([-100,0])
plt.legend()
plt.show()

# plt.figure(figsize=(15,3))
# plt.plot(time[:],X[0,:],"-",label="RK4 Solution for y")
# #plt.scatter(time[sp>0],sp[sp>0]+thresh0,marker="|",s=10000)

# plt.xlabel("t")
# plt.ylabel("X")
# #plt.ylim([-100,0])
# plt.legend()
# plt.show()

plt.figure(figsize=(15,3))

sp_times = np.arange(time.shape[0])[np.bool8(sp)]

plt.plot(sp_times[:-1],1000/np.diff(sp_times),"-",label="RK4 Solution for y")
#plt.scatter(time[sp>0],sp[sp>0]+thresh0,marker="|",s=10000)

plt.xlabel("t")
plt.ylabel("X")
#plt.ylim([-100,0])
plt.legend()
plt.show()




CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [35]:
np.bool8(sp)

array([False, False, False, ..., False, False, False])

In [26]:
plt.figure(figsize=(4,3))
data = []
for gB in np.linspace(0.,1,20):
    ep = 1e-4
    time = np.arange(0,10,ep)
    L = 0*np.float32(np.logical_and(time>1.2,time<2.2))

    kr = 1/0.3
    kf = 1/1.2

    Cm = 0.00144
    gL = 1.44
    Vl = -62
    VR =0

    gR = 20
    thresh0 = -55
    reset = -62

    tGamma = 0.2
    delth = 1

    #gB = 0.7
    gN = 0.04
    
    def f(y,t):
        Rs,V = y
        dRs = kr*L[int(t/ep)]*(1-Rs)-kf*Rs
        dV = 1/Cm*(-gL*(V-Vl)-gR*(V-VR)*Rs -gB*(V-VR)- gN*np.random.normal()*(V-VR))
        return np.array([dRs,dV])

    X = np.zeros((2,time.shape[0]))
    wth = np.zeros(time.shape[0])
    sp = np.zeros(time.shape[0])

    X[:,0]= [0,-60]

    for i in range(1,time.shape[0]):
        X[:,i] = X[:,i-1] + ep*f(X[:,i-1],time[i-1])
        wth[i] = wth[i-1]*np.exp(-ep/tGamma) 
        if X[1,i]>thresh0+wth[i]:
            X[1,i] = reset
            sp[i-1] = 1
            wth[i] = wth[i] + delth

    plt.scatter(gB,sp.sum()/10)
    data.append((gB,sp.sum()/10))
plt.xlabel('Baseline Conductance Input (nS)')
plt.ylabel('Firing Rate (Hz)')
plt.show()


KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import LinearRegression
data = np.array(data)
model = LinearRegression()

In [ ]:
model.fit(data[:,1].reshape(-1,1),data[:,0].reshape(-1,1))

In [ ]:
model.predict([[4]])

In [ ]:
plt.figure(figsize=(4,3))
data = []
for gR in np.linspace(0.,1,20):
    ep = 1e-3
    time = np.arange(0,10,ep)
    L = 1*np.float32(np.logical_and(time>1.2,time<2.2))

    kr = 1/0.3
    kf = 1/1.2

    Cm = 0.00144
    gL = 1.44
    Vl = -62
    VR =0

    #gR = 20
    thresh0 = -55
    reset = -62

    tGamma = 0.6
    delth = 0

    gB = model.predict([[4]])
    gN = 0.04
    
    def f(y,t):
        Rs,V = y
        dRs = kr*L[int(t/ep)]*(1-Rs)-kf*Rs
        dV = 1/Cm*(-gL*(V-Vl)-gR*(V-VR)*Rs -gB*(V-VR)- gN*np.random.normal()*(V-VR))
        return np.array([dRs,dV])

    X = np.zeros((2,time.shape[0]))
    wth = np.zeros(time.shape[0])
    sp = np.zeros(time.shape[0])

    X[:,0]= [0,-60]

    for i in range(1,time.shape[0]):
        X[:,i] = X[:,i-1] + ep*f(X[:,i-1],time[i-1])
        wth[i] = wth[i-1]*np.exp(-ep/tGamma) 
        if X[1,i]>thresh0+wth[i]:
            X[1,i] = reset
            sp[i-1] = 1
            wth[i] = wth[i] + delth

    plt.scatter(gR,sp.sum()/10)
    data.append((gR,sp.sum()/10))
plt.xlabel('Baseline Conductance Input (nS)')
plt.ylabel('Firing Rate (Hz)')
plt.show()


In [ ]:
%%time

ep = 1e-3
time = np.arange(0,10,ep)
L = 1*np.float32(np.logical_and(time>2,time<4))
n_rep = 5
data = []

Cm = 0.00144
gL = 1.44
Vl = -62
VR =0
thresh0 = -55
reset = -62
tGamma = 0.6
gN = 0.04
kr = 1/0.3
kf = 1/6

def f(y,t):
    Rs,V = y
    dRs = kr*L[int(t/ep)]*(1-Rs)-kf*Rs
    dV = 1/Cm*(-gL*(V-Vl)-gR*(V-VR)*Rs -gB*(V-VR)- gN*np.random.normal()*(V-VR))
    return np.array([dRs,dV])

for gB in np.linspace(0.,1,5):
    for gR in np.linspace(0.,1,5):
            for delth in np.linspace(0.,1,5):
                sps = []
                for i in range(n_rep):
                    X = np.zeros((2,time.shape[0]))
                    wth = np.zeros(time.shape[0])
                    sp = np.zeros(time.shape[0])
                    X[:,0]= [0,-60]
                    for i in range(1,time.shape[0]):
                        X[:,i] = X[:,i-1] + ep*f(X[:,i-1],time[i-1])
                        wth[i] = wth[i-1]*np.exp(-ep/tGamma) 
                        if X[1,i]>thresh0+wth[i]:
                            X[1,i] = reset
                            sp[i-1] = 1
                            wth[i] = wth[i] + delth
                    sps.append(sp)
                data.append((gB,gR,delth,np.mean(sps,axis=0)))              

In [ ]:
np.mean([data[0][3],data[1][3]],axis=0).shape

In [ ]:
for i in data[20:25]:
    plt.plot(time[:],i[3],"-")